In [1]:
import sys, os
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
from datetime import datetime

In [2]:
connection = pg.connect("host='192.168.1.8' dbname='holddb' user='pguser' password='ch@cky@b@s@bins'")

In [15]:
df = psql.read_sql("SELECT * FROM bookmark", connection)

In [10]:
# 1. This is the 1st recommended approach with explanation 
#"Create helper Series with compare by allergies. 
# Change order and use cumulative sum by Series.cumsum, 
# then it pass to GroupBy.cumcount and for second and third column compare by isin:"

s = df['mood'].eq('allergies').iloc[::-1].cumsum()
df = df[df.groupby(s).cumcount(ascending=False).isin([1,2])]
print (df)

      id    food_word_1    food_word_2    food_word_3  \
8      9          pasta                                 
9     10          pasta                                 
11    12          bread      olive oil         coffee   
17    18          bread      olive oil         coffee   
18    19          curry           rice         coffee   
21    22      cous cous  tomatoe sauce     chick peas   
22    23          bread         coffee      olive oil   
27    28           eggs         coffee                  
28    33           fish           pork       red wine   
50    55          bread      olive oil         coffee   
51    56           fish           rice         coffee   
63    68        chicken       potatoes     white wine   
64    69          bread         coffee                  
67    72       tortilla           tuna  tomatoe sauce   
70    75  vegtable soup     hard chese     soft chese   
71    76       red wine                                 
85    92     cereal bar        

In [13]:
# 2. This is the 2nd recommended approach (without explanation) 

s = df['mood'].eq('allergies').iloc[::-1].cumsum().sort_index()
df = df[(df.groupby(s).cumcount(ascending=False) < 3) & s.duplicated(keep='last')]
print (df)

      id    food_word_1    food_word_2    food_word_3  \
8      9          pasta                                 
9     10          pasta                                 
11    12          bread      olive oil         coffee   
17    18          bread      olive oil         coffee   
18    19          curry           rice         coffee   
21    22      cous cous  tomatoe sauce     chick peas   
22    23          bread         coffee      olive oil   
27    28           eggs         coffee                  
28    33           fish           pork       red wine   
50    55          bread      olive oil         coffee   
51    56           fish           rice         coffee   
63    68        chicken       potatoes     white wine   
64    69          bread         coffee                  
67    72       tortilla           tuna  tomatoe sauce   
70    75  vegtable soup     hard chese     soft chese   
71    76       red wine                                 
85    92     cereal bar        

In [16]:
# 3. This is an attempt to add some date driven data into the mix 

s = df['mood'].eq('allergies').iloc[::-1].cumsum()
df = df[df.groupby(s).cumcount(ascending=False).isin([0,1,2])]

# BUT IS WITHIN A 48 HR PERIOD OF VALUES IN 'S'

date_df = df # ['date'] < 48 s['date']
print (date_df['food_word_1'])

8              pasta
9              pasta
10             bread
11             bread
12            coffee
13              beef
14             pasta
15        cereal bar
16          red wine
17             bread
18             curry
19              nuts
21         cous cous
22             bread
23              duck
27              eggs
28              fish
29        cereal bar
50             bread
51              fish
52             salad
63           chicken
64             bread
65            fennel
66           almonds
67          tortilla
68        cereal bar
70     vegtable soup
71          red wine
72        cereal bar
73        cereal bar
85        cereal bar
86           chicken
87          pancakes
88             bread
89             salad
90              eggs
91              lamb
92           noodles
93             bread
117          gnocchi
118          peppers
119             eggs
128       croissante
129    tomatoe sauce
130            bread
175            bread
176        co

In [ ]:
if 'fennel' in df.values:
    print('Element exists in Dataframe')

In [17]:
df.shape

(49, 11)